In [1]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# Imports we'll be using
from tensorflow import keras
from keras.layers import *
import numpy as np

2024-02-28 21:05:50.348877: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df_import = pd.read_csv('./data_collection/output.csv')

In [4]:
df_import.head()

,n,gameOver,head,score,visionUp,visionRight,visionDown,visionLeft,directionChoice
0,0,0,"[8, 9]",0,0.0,0.0,0.0,1.0,3
1,1,0,"[8, 10]",0,0.0,0.0,0.0,1.0,3
2,2,0,"[8, 11]",0,0.0,0.0,0.0,1.0,0
3,3,0,"[7, 11]",0,0.0,0.0,1.0,0.0,2
4,4,0,"[6, 11]",0,0.0,0.0,1.0,0.0,3


In [5]:
df = df_import[['gameOver','visionUp','visionRight','visionDown','visionLeft','directionChoice']]

In [6]:
df.head()

,gameOver,visionUp,visionRight,visionDown,visionLeft,directionChoice
0,0,0.0,0.0,0.0,1.0,3
1,0,0.0,0.0,0.0,1.0,3
2,0,0.0,0.0,0.0,1.0,0
3,0,0.0,0.0,1.0,0.0,2
4,0,0.0,0.0,1.0,0.0,3


In [7]:
dummy_columns = pd.get_dummies(df['directionChoice'], prefix='directionChoice')

In [8]:
dummy_columns = dummy_columns.replace({True:1, False:0})

In [9]:
df = pd.concat([df, dummy_columns], axis=1)
df.drop('directionChoice', inplace=True, axis=1)

In [10]:
df.head()

,gameOver,visionUp,visionRight,visionDown,visionLeft,directionChoice_0,directionChoice_1,directionChoice_2,directionChoice_3
0,0,0.0,0.0,0.0,1.0,0,0,0,1
1,0,0.0,0.0,0.0,1.0,0,0,0,1
2,0,0.0,0.0,0.0,1.0,1,0,0,0
3,0,0.0,0.0,1.0,0.0,0,0,1,0
4,0,0.0,0.0,1.0,0.0,0,0,0,1


In [11]:
X = df[['directionChoice_0','directionChoice_1', 'directionChoice_2', 'directionChoice_3','visionUp', 'visionRight', 'visionDown', 'visionLeft']].to_numpy()
y = df['gameOver'].to_numpy()

1151437

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [65]:
# First, we initialize a sequential model
# Create a Sequential model
model = keras.Sequential()

# Add a single Dense layer with 5 input nodes and 1 output node
model.add(Input(shape=(8,)))
# model.add(Dense(units=(5), activation='linear'))
# model.add(Dense(units=(5), activation='linear'))
model.add(Dense(units=(1), activation='relu'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 9 (36.00 Byte)
Trainable params: 9 (36.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [66]:
# Train the model
model.fit(X_train, y_train, epochs=3, batch_size=32, verbose=1)

# Evaluate the model
out = model.evaluate(X_train, y_train)
print("Training accuracy:", out)

Epoch 1/3
28786/28786 [==============================] - 22s 745us/step - loss: 0.0308 - accuracy: 0.9636
Epoch 2/3
28786/28786 [==============================] - 22s 749us/step - loss: 0.0271 - accuracy: 0.9669
Epoch 3/3
28786/28786 [==============================] - 19s 669us/step - loss: 0.0271 - accuracy: 0.9685
Training accuracy: [0.027082795277237892, 0.9684589505195618]


In [68]:
model.save("./models/possible_mvp.keras")

In [67]:
loss, accuracy = model.evaluate(X_test, y_test)

7197/7197 [==============================] - 5s 689us/step - loss: 0.0271 - accuracy: 0.9684


In [62]:
test = X_test[0]
test = test.reshape((1,8))
test = np.array([0,1,0,0,0,0,0,1]).reshape((1,8))
model.predict(test)



1/1 [==============================] - 0s 20ms/step


array([[0.02088982]], dtype=float32)

In [44]:
test

array([[0., 1., 0., 0., 0., 1., 0., 0.]])

In [74]:
for i in range(0, 4):
    direction = [0, 0, 0, 0]
    direction[i] = 1
    a = np.array(direction + [0,1,1,1])
    print(a)



[1 0 0 0 0 1 1 1]
[0 1 0 0 0 1 1 1]
[0 0 1 0 0 1 1 1]
[0 0 0 1 0 1 1 1]


In [83]:
input = np.array([0,0,1,0,1,0,0,0])
input = input.reshape((1,8))
model.predict(input)

1/1 [==============================] - 0s 20ms/step


array([[0.]], dtype=float32)

In [76]:
input

array([[1, 0, 0, 0, 1, 0, 0, 0]])

In [94]:
len(df[(df['visionUp'] == 1) & (df['gameOver'] == 1)])

22283